## RBIF109 Assignment 3  
Author: Laura Wooten  

**(3.1) FASTQ to FASTA converter: Implement a FASTQ to FASTA file format converter in python (the input is a FASTQ file and the output is a FASTA file)**

Input files:  
TestX.fastq [1]

Output files:  
TestX.fa

Requirements:  
Python3 must be installed on your operating system.

Execute the following script in the terminal as depicted below, ensure current working directory includes fastq_fasta_converter.py, and TestX.fastq:    
`python3 fastq_fasta_converter.py TestX.fastq`

Steps:  
1. Assigns the input file to a variable.
2. Reads in the file line-by-line in groups of 4 where the first line begins with an "@" character.
3. Assigns the first and second lines to variables.
4. Skips over the remaining two lines in the group.
5. Writes the two variables to the output file, replacing the "@" of the first line with ">".
6. Continues to the next iteration until it reaches the end of file.



**(3.2) I have a sam file called “sequences.sam” and a genomic feature file called “features.bed”. I would like to annotate all the sequences in the sam file with the bed file (overlapping features). Can you write down the commands needed using samtools and bedtools?**

Steps:  
1. Use samtools [2] to convert file to BAM, sort, and index.
2. Use bedtools [3] to intersect BAM with BED. 

In [ ]:
#bash#
samtools view -b sequences.sam > sequences.bam
samtools sort sequences.bam -o sequences_sorted.bam
samtools index sequences_sorted.bam 
bedtools intersect -a sequences_sorted.bam -b features.bed > sequences_annotated.bam

**(3.3) Can you obtain at least two mRNA isoforms of TP53 from NCBI or other online resources? Generate a bed file containing both, color them differently and visualize them on UCSC genome browser? Take a screenshot of the visualization.**

> I did this two ways, once before starting week 8, then again after learning about BLAT. Wasn't sure which one you were looking for so I included both.

Steps for first image:  
1. Download isoforms [4], and genome assembly GRCh38.p14 chromosome 17 as the reference [5].
2. Unzip & index reference file
3. BWA to align to reference
4. Samtools to convert to BAM, sort, index
5. Bedtools to convert to BED
6. awk to add thickStart, thickEnd, and RBG
7. cat to join BED files.
8. Upload to UCSC [6] & add track title

In [ ]:
#bash#
gunzip Homo_sapiens.GRCh38.dna.chromosome.17.fa.gz
reference="Homo_sapiens.GRCh38.dna.chromosome.17.fa"
bwa index $reference

##isoforms##
# NM_001276695.3|:1-2572 Homo sapiens tumor protein p53 (TP53), transcript variant 4, mRNA
# NM_001407264.1|:1-2522 Homo sapiens tumor protein p53 (TP53), transcript variant 10, mRNA

##per isoform - choose differnt color##
bwa mem $reference isoform.fa > isoform.sam
samtools view -b isoform.sam > isoform.bam 
samtools sort isoform.bam -o isoform_sorted.bam
samtools index isoform_sorted.bam 
bedtools bamtobed -i isoform_sorted.bam > isoform.bed
awk '{OFS="\t"; print $1, $2, $3, $4, $5, $6, $2, $3, "0,255,0"}' isoform.bed > isoform_colored.bed

##join##
cat isoform1_colored.bed isoform2_colored.bed > combined_isoforms.bed

![alt text](./UCSC_Screenshots/UCSC_TP53isoforms.JPG)

Steps for second image:    
1. combine fasta files into one file
2. used BLAT to align to chromosome 17
3. convert to bed
4. awk to change column #9 to add color
5. Upload to UCSC [6] & add track title

In [ ]:
#bash#
cat NM_001276695.3.fa NM_001407264.1.fa > combinedTP53.fa
blat chr17.2bit combinedTP53.fa blat_output.psl
pslToBed blat_output.psl blat_output.bed
awk 'BEGIN{OFS="\t"} NR==1{$9="0,255,0"} NR==2{$9="0,0,255"} {print}' blat_output.bed > blat_colored.bed

![alt text](./UCSC_Screenshots/blat_UCSC_TP53isoforms.JPG)

References:  

1. Hartwig Medical Foundation. (n.d.). TestX.fastq renamed from TESTX_H7YRLADXX_S1_L001_R1_001.fastq. GitHub. Retrieved from https://github.com/hartwigmedical/testdata32. HTSlib. (n.d.). Samtools. Retrieved from https://www.htslib.org/doc/samtools.html4
3. Quinlan, A. R. (n.d.). Bedtools: Intersect. Retrieved from https://bedtools.readthedocs.io/en/latest/content/tools/intersect.html5
4. National Center for Biotechnology Information (NCBI). (n.d.). TP53 tumor protein p53 [Homo sapiens (human)]. Retrieved from https://www.ncbi.nlm.nih.gov/gene/7167
5. Ensembl. (n.d.). Homo sapiens DNA sequence. Retrieved from https://ftp.ensembl.org/pub/release-113/fasta/homo_sapiens/d7a/
6. UCSC Genome Browser. (n.d.). Retrieved from https://genome.ucsc.edu/index.html.html